In [1]:
import zipfile
import os
from os import path
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from skimage.transform import resize
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.applications import Xception
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Input
from sklearn import preprocessing
from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, classification_report, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from collections import Counter
import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)

In [6]:
external_df = pd.read_csv('~/Desktop/galvanize/predicting_engagement_v1/notebooks/csv/external_df.csv')
clean_df = pd.read_csv('~/Desktop/galvanize/predicting_engagement_v1/notebooks/csv/clean_df.csv')
external_df.drop(columns = ['Unnamed: 0', 'datetime'], axis = 1, inplace = True)

In [ ]:
#holdout
r_df = df.copy()
holdout = 0.2
mask = np.random.rand(len(df)) < (1-holdout)
train_val_df = r_df[mask]
test_df = r_df[~mask]

X = df.copy()
Y = df['new_engagement']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .2, random_state = seed, shuffle = True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#check classes
class_names_int = list(X_train['engagement_label'].unique())
class_names_int

In [ ]:
#image folder paths - MAKE SURE SHUTIL IS SET TO COPY AND NOT MOVE

train_images = '/home/ubuntu/images/regression/train/'
val_images = '/home/ubuntu/images/regression/val/'

#move image files to categories
for c in class_names_int:
    for i in list(X_train[X_train['engagement_label'] == c]['shortcode']): #image id
        try:
            get_image = os.path.join(r'/home/ubuntu/train_/' + str(c), i + '.jpg') #path to images
            move_image_to_cat = shutil.copy(get_image, '/home/ubuntu/images/regression/train/')
        except:
            pass

for c in class_names_int:
    for i in list(X_test[X_test['engagement_label'] == c]['shortcode']): #image id
        try:
            get_image = os.path.join(r'/home/ubuntu/train_/' + str(c), i + '.jpg') #path to images
            move_image_to_cat = shutil.copy(get_image, '/home/ubuntu/images/regression/val/')
        except:
            pass


! rm -rf train_val_images/.ipynb_checkpoints/
! rm -rf test_images/.ipynb_checkpoints/

In [ ]:
#filepath for flow_from_directory
X_train['directory_path'] = '/home/ubuntu/images/regression/train/' + X_train['shortcode'] + '.jpg'
X_test['directory_path'] = '/home/ubuntu/images/regression/val/' + X_test['shortcode'] + '.jpg'

#create a dataset
train_dir = '/home/ubuntu/images/regression/'
val_dir = '/home/ubuntu/images/regression/'
batch_size = 16
img_height = 299
img_width = 299

trainimggen = ImageDataGenerator(rescale = (1./255),
                                  horizontal_flip=True, 
                                  rotation_range=10, 
                                  width_shift_range=1, 
                                  height_shift_range=1,
                                  brightness_range=None,
                                  shear_range=1,
                                  zoom_range=1,
                                  channel_shift_range=2)
validationimggen = ImageDataGenerator(rescale = (1./255))


train_ds = trainimggen.flow_from_dataframe(dataframe = X_train,
                                           x_col = 'directory_path',
                                           y_col = 'new_engagement',
                                           class_mode = 'raw',
                                           target_size=(img_height,img_width), 
                                           batch_size=batch_size)

val_ds = validationimggen.flow_from_dataframe(dataframe = X_test,
                                           x_col = 'directory_path',
                                           y_col = 'new_engagement',
                                           class_mode = 'raw',
                                           target_size=(img_height,img_width), 
                                           batch_size=batch_size)

In [ ]:
#variables for batch size epoch step
ti = 4508
vi = 1128

In [ ]:
#model helper functions
num_classes = 1

def add_new_last_layer(base_model, nb_classes= num_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    # Get the output shape of the models last layer
    x = base_model.output
    # Convert final MxNxC tensor output into a 1xC tensor where C is the # of channels.
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(1024, activation='relu')(x) 
    x = keras.layers.Dropout(.5)(x)
    predictions = keras.layers.Dense(1, activation = 'linear')(x)
    model = keras.models.Model(inputs=base_model.input, outputs=predictions)
    return model

# This will freeze the weights on all the layers except for our new dense layer
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.01),
              loss=['mean_absolute_percentage_error'],
              metrics=['accuracy'])

def change_trainable_layers(model, trainable_index):
    for layer in model.layers[:trainable_index]:
        layer.trainable = False
    for layer in model.layers[trainable_index:]:
        layer.trainable = True
        
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
project_name = "transfer_learning"
tensorboard_cb = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
#load vgg19 as base model
vgg19 = keras.applications.VGG19(include_top=False, 
                                      weights='imagenet', input_shape=(img_height,img_width,3))

In [ ]:
vgg19_model = add_new_last_layer(vgg19)
setup_to_transfer_learn(vgg19_model,vgg19)

In [ ]:
#run last layer
training_images = ti
steps_p_epoch = training_images / batch_size
validation_images = vi
validation_steps = validation_images / batch_size

vgg19_model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.01),
              loss=['mean_squared_error'],
              metrics=['mae'])

model_cp= ModelCheckpoint(filepath='/home/ubuntu/models/vgg19_model.hdf5',
                            save_best_only=True)

vgg19_model_history = vgg19_model.fit(train_ds, 
                validation_data=val_ds,
                epochs=10,
                callbacks=[model_cp, tensorboard_cb])

In [ ]:
#visualize training results

mse = vgg19_model_history.history['mse']
val_mse = vgg19_model_history.history['val_mse']
loss = vgg19_model_history.history['loss']
val_loss = vgg19_model_history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.plot(epochs_range, mse, label = 'Training Accuracy')
plt.plot(epochs_range, val_mse, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.autoscale()
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.autoscale()
plt.show();

In [ ]:
#confirming that we are unfreezing block5_pool - 2M params
change_trainable_layers(vgg19_model, 20)
# vgg19_model.summary()
# for i, layer in enumerate(vgg19_model.layers):
#     print(i, layer.name, layer.trainable)

In [ ]:
#run last layer
training_images = ti
steps_p_epoch = training_images / batch_size
validation_images = vi
validation_steps = validation_images / batch_size

vgg19_model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001),
              loss=['mean_squared_error'],
              metrics=['mae'])

model_cp= ModelCheckpoint(filepath='/home/ubuntu/models/vgg19_model.hdf5',
                            save_best_only=True)

vgg19_model_2_history = vgg19_model.fit(train_ds, 
                validation_data=val_ds,
                epochs=20,
                callbacks=[model_cp, tensorboard_cb])

In [ ]:
#xception model
xception = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(img_height,img_width,3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
x_model = add_new_last_layer(xception)
setup_to_transfer_learn(x_model,xception)

In [ ]:
#run last layer
training_images = ti
steps_p_epoch = training_images / batch_size
validation_images = vi
validation_steps = validation_images / batch_size

x_model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.01),
              loss=['mean_absolute_percentage_error'],
              metrics=['mse'])

model_cp= ModelCheckpoint(filepath='/home/ubuntu/models/x_model.hdf5',
                            save_best_only=True)

x_model_history = vgg19_model.fit(train_ds, 
                validation_data=val_ds,
                epochs=10,
                callbacks=[model_cp, tensorboard_cb])

In [ ]:
change_trainable_layers(x_model, 129)
# x_model.summary()

In [ ]:
#run last layer
training_images = ti
steps_p_epoch = training_images / batch_size
validation_images = vi
validation_steps = validation_images / batch_size

x_model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001),
              loss=['mean_absolute_percentage_error'],
              metrics= ['mse'])

model_cp= ModelCheckpoint(filepath='/home/ubuntu/models/x_model.hdf5',
                            save_best_only=True)

x_model_2_history = x_model.fit(train_ds, 
                validation_data=val_ds,
                epochs=20,
                callbacks=[model_cp, tensorboard_cb])

In [ ]:
#visualize training results

mse = x_model_2_history.history['mse']
val_mse = x_model_2_history.history['val_mse']
mape = x_model_2_history.history['loss']
val_mape = x_model_2_history.history['val_loss']

epochs_range = range(20)

plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.plot(epochs_range, mse, label = 'Training MSE')
plt.plot(epochs_range, val_mse, label = 'Validation MSE')
plt.legend(loc = 'upper right')
plt.autoscale()
plt.title('Training and Validation MSE')

plt.subplot(1,2,2)
plt.plot(epochs_range, mape, label = 'Training Loss (MAPE)')
plt.plot(epochs_range, val_mape, label = 'Validation Loss (MAPE)')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.autoscale()
plt.savefig('/home/ubuntu/graphs/xception_train_val_mse_mape.png')
plt.show();

In [ ]:
# calculate mean score for AVA dataset
def mean_score(scores):
    si = np.arange(1, 11, 1)
    mean = np.sum(scores * si)
    return mean

# calculate standard deviation of scores for AVA dataset
def std_score(scores):
    si = np.arange(1, 11, 1)
    mean = mean_score(scores)
    std = np.sqrt(np.sum(((si - mean) ** 2) * scores))
    return std

In [ ]:
#mobilenet
base_model = MobileNet((img_height, img_width, 3), alpha = 1, include_top=False, pooling= 'avg')
for layer in base_model.layers:
    layer.trainable = False

x = layers.Dropout(0.75)(base_model.output)
x = layers.Dense(10, activation='softmax')(x)

mna_model = Model(base_model.input, x)
mna_model.load_weights('/home/ubuntu/weights/weights_mobilenet_aesthetic_0.07.hdf5')

In [ ]:
from tensorflow.keras import backend as K
def earth_mover_loss(y_true, y_pred):
    cdf_ytrue = K.cumsum(y_true, axis=-1)
    cdf_ypred = K.cumsum(y_pred, axis=-1)
    samplewise_emd = K.sqrt(K.mean(K.square(K.abs(cdf_ytrue - cdf_ypred)), axis=-1))
    return K.mean(samplewise_emd)

In [ ]:
#run last layer
training_images = ti
steps_p_epoch = training_images / batch_size
validation_images = vi
validation_steps = validation_images / batch_size

mna_model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001),
              loss=earth_mover_loss,
              metrics= ['mse'])

model_cp= ModelCheckpoint(filepath='/home/ubuntu/models/mna_model.hdf5',
                            save_best_only=True)

mna_model_history = mna_model.fit(train_ds, 
                validation_data=val_ds,
                epochs=20,
                callbacks=[model_cp, tensorboard_cb])

In [ ]:
#load image for predictions
fp = train_ds.filepaths[850]
img = tf.keras.preprocessing.image.load_img(fp, (299,299))
plt.imshow(img);

In [ ]:
img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
img=tf.keras.preprocessing.image.img_to_array(img)
predictions=mn_model_aesthetic.predict(np.array([img]))
scores = mn_model_aesthetic.predict(np.array([img]), verbose=0)[0]
mean_score(scores), std_score(scores)

In [ ]:
#generating predictions - save in pandas dataframe
tfp = train_ds.filepaths
vfp = val_ds.filepaths

mn_aesthetic = pd.DataFrame()
scores_list = []
shortcodes = []
std_list = []
filepath = []

#loop over training
for fp in tfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    scores = mna_model.predict(np.array([img]), verbose=0)[0]
    mean = mean_score(scores)
    std = std_score(scores)
    shortcode = fp.split('/')[-1][:-4]
    scores_list.append(mean)
    std_list.append(std)
    shortcodes.append(shortcode)
    fp = '/home/ubuntu/images/regression/train/' + shortcode + '.jpg'
    filepath.append(fp)

#loop over val
for fp in vfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    scores = mna_model.predict(np.array([img]), verbose=0)[0]
    mean = mean_score(scores)
    std = std_score(scores)
    shortcode = fp.split('/')[-1][:-4]
    scores_list.append(mean)
    std_list.append(std)
    shortcodes.append(shortcode)
    fp = '/home/ubuntu/images/regression/val/' + shortcode + '.jpg'
    filepath.append(fp)
    
mn_aesthetic['shortcodes'] = shortcodes
mn_aesthetic['mean_aesthetic'] = scores_list
mn_aesthetic['std_aesthetic'] = std_list
mn_aesthetic['filepath'] = filepath

mn_aesthetic.to_csv('/home/ubuntu/csv/mn_aesthetic.csv')

In [ ]:
#mobilenet
mobilenet_model = MobileNet((None, None, 3), alpha=1, include_top=False, pooling='avg', weights=None)
x = layers.Dropout(0.75)(mobilenet_model.output)
x = layers.Dense(10, activation='softmax')(x)
    
mn_model_technical = Model(mobilenet_model.input, x)
mn_model_technical.load_weights('/home/ubuntu/weights/weights_mobilenet_technical_0.11.hdf5')

In [ ]:
#generating predictions
tfp = train_ds.filepaths
vfp = val_ds.filepaths

mn_technical = pd.DataFrame()
scores_list = []
shortcodes = []
std_list = []
filepath = []

#loop over training
for fp in tfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    scores = mn_model_technical.predict(np.array([img]), verbose=0)[0]
    mean = mean_score(scores)
    std = std_score(scores)
    shortcode = fp.split('/')[-1][:-4]
    scores_list.append(mean)
    std_list.append(std)
    shortcodes.append(shortcode)
    fp = '/home/ubuntu/images/regression/val/' + shortcode + '.jpg'
    filepath.append(fp)

#loop over val
for fp in vfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    scores = mn_model_technical.predict(np.array([img]), verbose=0)[0]
    mean = mean_score(scores)
    std = std_score(scores)
    shortcode = fp.split('/')[-1][:-4]
    scores_list.append(mean)
    std_list.append(std)
    shortcodes.append(shortcode)
    fp = '/home/ubuntu/images/regression/val/' + shortcode + '.jpg'
    filepath.append(fp)

mn_technical['shortcodes'] = shortcodes
mn_technical['mean_aesthetic'] = scores_list
mn_technical['std_aesthetic'] = std_list
mn_technical['filepath'] = filepath

mn_technical.to_csv('/home/ubuntu/csv/mn_technical.csv')

In [ ]:
mn_technical.rename(columns = {'mean_aesthetic': 'mean_technical', 'std_aesthetic': 'std_technical'}, inplace = True)
mn_technical.sort_values(by = 'mean_technical', ascending = False, inplace = True)
mn_technical.to_csv('/home/ubuntu/csv/mn_technical.csv')
mn_aesthetic.sort_values(by = 'mean_aesthetic', ascending = False, inplace = True)
mn_aesthetic

In [ ]:
#xception model for feature extraction
x_obj_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=True)  # Do not include the ImageNet classifier at the top.

In [ ]:
#generating predictions
tfp = train_ds.filepaths
vfp = val_ds.filepaths
top = 3
x_features = pd.DataFrame()

shortcodes = []
keys = []
values = []

#loop over training
for fp in tfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    img=tf.keras.applications.xception.preprocess_input(img)
    predictions=x_obj_model.predict(np.array([img]))
    decode = decode_predictions(predictions,top= top)
    shortcode = fp.split('/')[-1][:-4]
    shortcodes.append(shortcode)
    tkeys = []
    tvalues = []
    for d in decode[0]:
        key = d[1]
        tkeys.append(key)
        value = d[2]
        tvalues.append(value)
    keys.append(tkeys)
    values.append(tvalues)

for fp in vfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    img=tf.keras.applications.xception.preprocess_input(img)
    predictions=x_obj_model.predict(np.array([img]))
    decode = decode_predictions(predictions,top= top)
    shortcode = fp.split('/')[-1][:-4]
    shortcodes.append(shortcode)
    vkeys = []
    vvalues = []
    for d in decode[0]:
        key = d[1]
        vkeys.append(key)
        value = d[2]
        vvalues.append(value)
    keys.append(vkeys)
    values.append(vvalues)

x_features['shortcode'] = shortcodes
x_features['keys'] = keys
x_features['values'] = values

x_features.to_csv('/home/ubuntu/csv/x_features.csv')

In [ ]:
#generating predictions - save Xception predictions into pandas dataframe
tfp = train_ds.filepaths
vfp = val_ds.filepaths
x_predict = pd.DataFrame()

shortcodes = []
predictions = []

#loop over training
for fp in tfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    img=tf.keras.applications.xception.preprocess_input(img)
    predictions.append(x_model_3.predict(np.array([img])))
    shortcode = fp.split('/')[-1][:-4]
    shortcodes.append(shortcode)

for fp in vfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    img=tf.keras.applications.xception.preprocess_input(img)
    predictions.append(x_model_3.predict(np.array([img])))
    shortcode = fp.split('/')[-1][:-4]
    shortcodes.append(shortcode)

x_predict['shortcode'] = shortcodes
x_predict['predictions'] = predictions

x_predict.to_csv('/home/ubuntu/csv/x_predict.csv')

In [ ]:
vgg19_model = load_model('/home/ubuntu/models/vgg19_model.hdf5')

In [ ]:
#generating predictions - save predictions for vgg19 into pandas dataframe
tfp = train_ds.filepaths
vfp = val_ds.filepaths
vgg19_predict = pd.DataFrame()

shortcodes = []
predictions = []

#loop over training
for fp in tfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    img=tf.keras.applications.xception.preprocess_input(img)
    predictions.append(x_model_3.predict(np.array([img])))
    shortcode = fp.split('/')[-1][:-4]
    shortcodes.append(shortcode)

for fp in vfp:
    img=tf.keras.preprocessing.image.load_img(fp,target_size=(299,299))
    img=tf.keras.preprocessing.image.img_to_array(img)
    img=tf.keras.applications.xception.preprocess_input(img)
    predictions.append(x_model_3.predict(np.array([img])))
    shortcode = fp.split('/')[-1][:-4]
    shortcodes.append(shortcode)

vgg19_predict['shortcode'] = shortcodes
vgg19_predict['predictions'] = predictions

vgg19_predict.to_csv('/home/ubuntu/csv/vgg19_predict.csv')

In [ ]:
#for image classification - src not available in this file
pred = x_train_model.predict(val_ds).argmax(axis = 1)
y_true = val_ds.labels
#confusion matrix for classification
mat = confusion_matrix(y_true, pred)
fig, ax = plt.subplots(figsize = (10,10))
sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False)
plt.show();

In [ ]:
target_names = ['class 0', 'class 1', 'class 2', 'class 3']

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('model', MultinomialNB()),
              ])
nb.fit(X_train, y_train);

In [ ]:
%%time

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=target_names))

In [ ]:
count_vect = CountVectorizer(lowercase=True, tokenizer=None, stop_words='english',
                             analyzer='word', max_df=1.0, min_df=1,
                             max_features=None, ngram_range = (1,2))

count_vect.fit(X_train, y_train)

target_names = target_names

In [ ]:
X_train_counts = count_vect.transform(X_train)
print("The type of X_train_counts is {0}.".format(type(X_train_counts)))
print("The X matrix has {0} rows (documents) and {1} columns (words).".format(
        X_train_counts.shape[0], X_train_counts.shape[1]))

In [ ]:
tfidf_transformer = TfidfTransformer(use_idf=True)
tfidf_transformer.fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)

X_test_counts = count_vect.transform(X_test)
tfidf_transformer.fit(X_test_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [ ]:
nb_model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nb_model.fit(X_train_tfidf, y_train);

In [ ]:
%%time

y_pred = nb_model.predict(X_test_tfidf)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=target_names))

In [ ]:
mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize = (10,10))
sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False)
plt.show();

In [ ]:
feature_words = count_vect.get_feature_names()
n = 10 #number of top words associated with the category that we wish to see

for cat in range(4):    
    print(f"\nTarget: {cat}, name: {target_names[cat]}")
    log_prob = nb_model.feature_log_prob_[cat]
    i_topn = np.argsort(log_prob)[::-1][:n]
    features_topn = [feature_words[i] for i in i_topn]
    print(f"Top {n} tokens: ", features_topn)